In [16]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [17]:
from typing import List

from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage, ToolMessage


from pydantic import BaseModel, Field
from tempfile import TemporaryDirectory
from langchain_community.agent_toolkits import FileManagementToolkit

In [18]:
# Get the file management tools and assign them to variables
toolkit = FileManagementToolkit(
    selected_tools=["read_file", "write_file", "list_directory"]
)
tools = toolkit.get_tools()
# Assign the tools to variables for direct access
read_file = next(tool for tool in tools if tool.name == "read_file")
write_file = next(tool for tool in tools if tool.name == "write_file")
list_directory = next(tool for tool in tools if tool.name == "list_directory")

In [19]:
from pydantic import BaseModel
class HighLevelDesign(BaseModel):
    """Instructions on how to prompt the LLM."""
    sector: str
    drivers: str
    functionalities: str
    non_functional_requirement: str
    channels: str
    application_type: str
    cloud: str
    architecture_style: str
    technology_choice: str
    architecture_decisions: str
    others: str

llm = ChatGroq(model="qwen-2.5-32b")
llm_with_tool = llm.bind_tools([HighLevelDesign])

In [20]:
LAPrompt = """ Define prompt to gather identified data points"""
def get_messages_info(state):
    srs = "SRS:\n" + state['srs'][-1].content
    last_message = "User Query:\n" + state['messages'][-1].content
    
    return[SystemMessage(content=LAPrompt), 
        HumanMessage(content=srs )] + state['messages']


def information_gathering(state):
    messages = get_messages_info(state["messages"])
    hld = state.get('hld')

    if hld:
        pass
    else:
        response = llm_with_tool.invoke(messages)
        return {"messages": [response]}

In [21]:
def is_clarified(state):
    messages = state["messages"]
    hld = state.get('hld')
    if isinstance(messages[-1], AIMessage) and messages[-1].tool_calls:
        return "yes" #"conclude_conversation"
    else:
        return "no" #"continue_conversation"

def conclude_conversation(state):
    return {
        "messages": [
            ToolMessage(
                content="Clarified and proceeding further",
                tool_call_id=state["messages"][-1].tool_calls[0]["id"],
            )
        ]
    }

In [22]:
SAPrompt = '''
Prompt to generate HLD
'''

SAReviewPrompt = '''
Prompt to revise HLD based on review comments or feedback
'''

def get_prompt_messages(messages: list, state):
    tool_call = None
    other_msgs = []
    
    for m in messages:
        if isinstance(m, AIMessage) and m.tool_calls:
            tool_call = m.tool_calls[0]["args"]
        elif isinstance(m, ToolMessage):
            continue
        elif tool_call is not None:
            other_msgs.append(m)

    iteration = state['iteration']
    format = state['hld_format'][-1].content
    last_message = state['messages'][-1].content
    srs = state['srs'][-1].content
    
    print("***** Revision Number ****", iteration)
    
    hld = state.get('hld')
    if hld:
        
        hld = state['hld'][-1].content
        
        return[
            SystemMessage(content=SAReviewPrompt.format(review=last_message, format=format)),
            HumanMessage(content=hld + srs)]
    else:
        return[
            SystemMessage(content=SAPrompt.format(reqs=tool_call, format=format)),
            HumanMessage(content=srs) ] + other_msgs 

def generate_hld(state):
    messages = get_prompt_messages(state["messages"], state)
    
    response = llm.invoke(messages)
    iteration = str(state['iteration'])
    file_name = "output/hld v" + str(iteration) +".md"
    
    write_file.invoke({"file_path": file_name, "text": response.content})
    
    return {"messages": [response], 
            "hld": [response]        
    }

In [23]:
critique = """
Prompt to review the HLD and provide feedback for revision
"""

def get_feedback_info(hld, srs):
    return [SystemMessage(content=critique),
           HumanMessage(content="SRS:\n" + srs[-1].content + "Current HLD:\n" + hld[-1].content)] 

llm = ChatGroq(model="qwen-2.5-32b", temperature=0, max_retries=1)

def reviewer(state):
    hld = state['hld']
    srs = state['srs']
    messages = get_feedback_info(hld, srs)

    response = llm.invoke(messages)

    iteration = str(state['iteration'])
    file_name = "output/hld_feedback v" + str(iteration) +".md"
    
    write_file.invoke({"file_path": file_name, "text": response.content})
    
    max_iteration = state['max_iteration']
    iteration = state['iteration'] + 1
    
    return {
        "messages": [response],
        "iteration": iteration
    }

def is_reviewed(state):
    max_iteration = state['max_iteration']
    iteration = state['iteration']
    last_message = state['messages'][-1].content
   
    if "Satisfied" in last_message.lower():
        return 'reviewed'
    elif iteration > max_iteration:
        return 'reviewed'
    else:
        return 'enhance'

In [24]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import END

class State(TypedDict):
    messages: Annotated[list, add_messages]
    srs: Annotated[list, add_messages]
    hld: Annotated[list, add_messages]
    hld_format: Annotated[list, add_messages]
    max_iteration: int
    iteration: int
    
    
memory = MemorySaver()
workflow = StateGraph(State)

workflow.add_edge(START, "information_gathering")

workflow.add_node("information_gathering", information_gathering)
workflow.add_node("generate_hld", generate_hld)
workflow.add_node("conclude_conversation", conclude_conversation)
workflow.add_node("reviewer",reviewer)

workflow.add_conditional_edges(
    "information_gathering", 
    is_clarified, 
    {"yes": "conclude_conversation",  "no": END}
)

workflow.add_conditional_edges(
    "reviewer", 
    is_reviewed, 
    {"reviewed": END,  "enhance": "generate_hld"}
)

workflow.add_edge("conclude_conversation", "generate_hld")
workflow.add_edge("generate_hld", "reviewer")

graph = workflow.compile(checkpointer=memory)

In [29]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

In [30]:
thread = {"configurable": {"thread_id": 4}}

# Ensure the input directory exists and has the required files
os.makedirs("input", exist_ok=True)

# Check if the files exist, if not create placeholder files
if not os.path.exists("input/srs.md"):
    with open("input/srs.md", "w") as f:
        f.write("# Software Requirements Specification\nThis is a placeholder SRS file.")
        
if not os.path.exists("input/hld_format.md"):
    with open("input/hld_format.md", "w") as f:
        f.write("# High Level Design Format\nThis is a placeholder HLD format file.")

SRS = read_file.invoke({"file_path": "input/srs.md"})
FORMAT = read_file.invoke({"file_path": "input/hld_format.md"})

# Ensure output directory exists
os.makedirs("output", exist_ok=True)

count = 1

while True:
    user = input("User (q/Q to quit): ")
    if user.lower() in ["quit", "q", "Q"]:
        print("AI: Byebye")
        break
    
    user_msg = user 
    
    try:
        for output in graph.stream(
            {
                "messages": [HumanMessage(content=user_msg)],
                "srs": [HumanMessage(content=SRS)],
                "hld_format": [HumanMessage(content=FORMAT)],
                "iteration": 1,
                "max_iteration": 2,
            }, 
            config=thread, 
            stream_mode="updates"):
            
            for key, value in output.items():
                print(f"***** Result from Agent: {key}")
                
                try:
                    # Get the last message from the current output
                    messages = value.get("messages", [])
                    if messages:
                        last_message = messages[-1]
                        print(f"Message content: {last_message.content}")
                except Exception as e:
                    print(f"Error displaying message: {str(e)}")
    except Exception as e:
        print(f"Error in graph execution: {str(e)}")

Error in graph execution: list indices must be integers or slices, not str
Error in graph execution: list indices must be integers or slices, not str
Error in graph execution: list indices must be integers or slices, not str
Error in graph execution: list indices must be integers or slices, not str
Error in graph execution: list indices must be integers or slices, not str
Error in graph execution: list indices must be integers or slices, not str
Error in graph execution: list indices must be integers or slices, not str
Error in graph execution: list indices must be integers or slices, not str
Error in graph execution: list indices must be integers or slices, not str
Error in graph execution: list indices must be integers or slices, not str
Error in graph execution: list indices must be integers or slices, not str
Error in graph execution: list indices must be integers or slices, not str
AI: Byebye


In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
from typing import List

from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage, ToolMessage

# Import the needed tools
from pydantic import BaseModel, Field
from tempfile import TemporaryDirectory
from langchain_community.agent_toolkits import FileManagementToolkit

# Get the file management tools and assign them to variables
toolkit = FileManagementToolkit(
    selected_tools=["read_file", "write_file", "list_directory"]
)
tools = toolkit.get_tools()
# Assign the tools to variables for direct access
read_file = next(tool for tool in tools if tool.name == "read_file")
write_file = next(tool for tool in tools if tool.name == "write_file")
list_directory = next(tool for tool in tools if tool.name == "list_directory")

from pydantic import BaseModel
class HighLevelDesign(BaseModel):
    """Instructions on how to prompt the LLM."""
    sector: str
    drivers: str
    functionalities: str
    non_functional_requirement: str
    channels: str
    application_type: str
    cloud: str
    architecture_style: str
    technology_choice: str
    architecture_decisions: str
    others: str

llm = ChatGroq(model="qwen-2.5-32b")
llm_with_tool = llm.bind_tools([HighLevelDesign])
LAPrompt = """ Define prompt to gather identified data points"""

# Fixed function - the issue is here
def get_messages_info(state):
    # The 'srs' and 'messages' keys contain lists, so we need to access them correctly
    srs = "SRS:\n" + state['srs'][0].content  # Access the first element with index 0
    
    # Add the user's message
    messages = state['messages']  # This is already a list
    
    return [SystemMessage(content=LAPrompt), 
           HumanMessage(content=srs)] + messages

def information_gathering(state):
    messages = get_messages_info(state)  # Pass the entire state
    hld = state.get('hld')

    if hld:
        pass
    else:
        response = llm_with_tool.invoke(messages)
        return {"messages": [response]}
        
def is_clarified(state):
    messages = state["messages"]
    hld = state.get('hld')
    if isinstance(messages[-1], AIMessage) and hasattr(messages[-1], 'tool_calls') and messages[-1].tool_calls:
        return "yes" #"conclude_conversation"
    else:
        return "no" #"continue_conversation"

def conclude_conversation(state):
    # Make sure tool_calls exists and has items
    if (isinstance(state["messages"][-1], AIMessage) and 
        hasattr(state["messages"][-1], 'tool_calls') and 
        state["messages"][-1].tool_calls):
        return {
            "messages": [
                ToolMessage(
                    content="Clarified and proceeding further",
                    tool_call_id=state["messages"][-1].tool_calls[0]["id"],
                )
            ]
        }
    else:
        # Fallback if no tool_calls are available
        return {
            "messages": [
                AIMessage(content="Proceeding to generate HLD")
            ]
        }
    
SAPrompt = '''
Prompt to generate HLD
'''

SAReviewPrompt = '''
Prompt to revise HLD based on review comments or feedback
'''

def get_prompt_messages(messages: list, state):
    tool_call = None
    other_msgs = []
    
    for m in messages:
        if isinstance(m, AIMessage) and hasattr(m, 'tool_calls') and m.tool_calls:
            tool_call = m.tool_calls[0]["args"]
        elif isinstance(m, ToolMessage):
            continue
        elif tool_call is not None:
            other_msgs.append(m)

    iteration = state['iteration']
    format = state['hld_format'][0].content  # Changed from [-1] to [0]
    
    # Get the last message content
    if state['messages']:
        last_message = state['messages'][-1].content
    else:
        last_message = ""
        
    srs = state['srs'][0].content  # Changed from [-1] to [0]
    
    print("***** Revision Number ****", iteration)
    
    hld = state.get('hld')
    if hld and hld:  # Make sure hld list has items
        hld_content = hld[0].content  # Changed from [-1] to [0]
        
        return [
            SystemMessage(content=SAReviewPrompt.format(review=last_message, format=format)),
            HumanMessage(content=hld_content + srs)]
    else:
        return [
            SystemMessage(content=SAPrompt.format(reqs=tool_call or {}, format=format)),
            HumanMessage(content=srs)] + other_msgs 

def generate_hld(state):
    messages = get_prompt_messages(state["messages"], state)
    
    response = llm.invoke(messages)
    iteration = str(state['iteration'])
    file_name = "output/hld v" + str(iteration) +".md"
    
    # Create the output directory if it doesn't exist
    os.makedirs("output", exist_ok=True)
    
    write_file.invoke({"file_path": file_name, "text": response.content})
    
    return {"messages": [response], 
            "hld": [response]        
    }
    
critique = """
Prompt to review the HLD and provide feedback for revision
"""

def get_feedback_info(hld, srs):
    # Access the first element in the lists
    hld_content = hld[0].content if hld else ""
    srs_content = srs[0].content if srs else ""
    
    return [SystemMessage(content=critique),
           HumanMessage(content="SRS:\n" + srs_content + "Current HLD:\n" + hld_content)] 

llm = ChatGroq(model="qwen-2.5-32b", temperature=0, max_retries=1)

def reviewer(state):
    hld = state.get('hld', [])
    srs = state.get('srs', [])
    
    # Check if the lists have items
    if not hld or not srs:
        return {
            "messages": [AIMessage(content="Missing HLD or SRS content")],
            "iteration": state['iteration'] + 1
        }
    
    messages = get_feedback_info(hld, srs)

    response = llm.invoke(messages)

    iteration = str(state['iteration'])
    file_name = "output/hld_feedback v" + str(iteration) +".md"
    
    write_file.invoke({"file_path": file_name, "text": response.content})
    
    max_iteration = state['max_iteration']
    iteration = state['iteration'] + 1
    
    return {
        "messages": [response],
        "iteration": iteration
    }

def is_reviewed(state):
    max_iteration = state['max_iteration']
    iteration = state['iteration']
    
    # Make sure there are messages before accessing the last one
    if state['messages']:
        last_message = state['messages'][-1].content
        if "Satisfied" in last_message.lower():
            return 'reviewed'
    
    if iteration > max_iteration:
        return 'reviewed'
    else:
        return 'enhance'
        
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import END

class State(TypedDict):
    messages: Annotated[list, add_messages]
    srs: Annotated[list, add_messages]
    hld: Annotated[list, add_messages]
    hld_format: Annotated[list, add_messages]
    max_iteration: int
    iteration: int
    
memory = MemorySaver()
workflow = StateGraph(State)

workflow.add_edge(START, "information_gathering")

workflow.add_node("information_gathering", information_gathering)
workflow.add_node("generate_hld", generate_hld)
workflow.add_node("conclude_conversation", conclude_conversation)
workflow.add_node("reviewer", reviewer)

workflow.add_conditional_edges(
    "information_gathering", 
    is_clarified, 
    {"yes": "conclude_conversation",  "no": END}
)

workflow.add_conditional_edges(
    "reviewer", 
    is_reviewed, 
    {"reviewed": END,  "enhance": "generate_hld"}
)

workflow.add_edge("conclude_conversation", "generate_hld")
workflow.add_edge("generate_hld", "reviewer")

graph = workflow.compile(checkpointer=memory)

# Ensure the input directory exists and has the required files
os.makedirs("input", exist_ok=True)

# Check if the files exist, if not create placeholder files
if not os.path.exists("input/srs.md"):
    with open("input/srs.md", "w") as f:
        f.write("# Software Requirements Specification\nThis is a placeholder SRS file.")
        
if not os.path.exists("input/hld_format.md"):
    with open("input/hld_format.md", "w") as f:
        f.write("# High Level Design Format\nThis is a placeholder HLD format file.")

SRS = read_file.invoke({"file_path": "input/srs.md"})
FORMAT = read_file.invoke({"file_path": "input/hld_format.md"})

# Ensure output directory exists
os.makedirs("output", exist_ok=True)

count = 1

while True:
    user = input("User (q/Q to quit): ")
    if user.lower() in ["quit", "q", "Q"]:
        print("AI: Byebye")
        break
    
    user_msg = user 
    
    try:
        # Initialize the state with appropriate values
        initial_state = {
            "messages": [HumanMessage(content=user_msg)],
            "srs": [HumanMessage(content=SRS)],
            "hld_format": [HumanMessage(content=FORMAT)],
            "iteration": 1,
            "max_iteration": 2,
        }
        
        for output in graph.stream(
            initial_state, 
            config=thread, 
            stream_mode="updates"):
            
            # Print each output node result
            print(f"***** Node completed: {list(output.keys())[0] if output else 'Unknown'}")
            
            # Try to extract and print the message content
            for key, value in output.items():
                if "messages" in value and value["messages"]:
                    last_message = value["messages"][-1]
                    print(f"Message: {last_message.content[:100]}...")  # Print first 100 chars
    except Exception as e:
        print(f"Error in graph execution: {str(e)}")
        import traceback
        traceback.print_exc()  # Print detailed error information

***** Node completed: information_gathering
Message: ...
***** Node completed: conclude_conversation
Message: Clarified and proceeding further...
***** Revision Number **** 1
***** Node completed: generate_hld
Message: To generate a High-Level Design (HLD) document, we first need to understand the scope and requiremen...
***** Node completed: reviewer
Message: The provided HLD template is a good starting point, but it can be refined and expanded to better ali...
***** Revision Number **** 2
***** Node completed: generate_hld
Message: To revise the High-Level Design (HLD) document based on review comments or feedback, you need to fol...
***** Node completed: reviewer
Message: The provided HLD template is a good starting point, but it can be refined and made more specific bas...


In [ ]:
"""
# Software Requirements Specification for E-commerce Platform

## Introduction
This SRS outlines the requirements for a new e-commerce platform focused on selling electronics.

## Functional Requirements
- User registration and authentication
- Product catalog with categories and search
- Shopping cart functionality
- Secure checkout process
- Order tracking
- Customer reviews
- Admin dashboard for inventory management

## Non-Functional Requirements
- High availability (99.99% uptime)
- Response time under 2 seconds
- Support for 10,000 concurrent users
- GDPR compliance
- Secure payment processing

## Technical Constraints
- Must use cloud infrastructure
- Mobile-first approach
- Integration with existing ERP system
"""